In [1]:
"""
Reads VIIRS NetCDF files (active fire and geolocation data)
Converts the swath to grid using geolocation information
Creates a geolocated active fire product representing maximum FRP for the duration of a fire event

Author: maxwell.cook@colorado.edu
"""

# Import packages

import os, shutil, time, glob, warnings, math
import earthaccess
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import h5py
import pyproj
import xarray as xr
import numpy as np
import gc
import traceback

from netCDF4 import Dataset
from matplotlib import pyplot as plt
from affine import Affine
from pyresample import geometry as geom
from pyresample import kd_tree as kdt
from os.path import join
from osgeo import gdal, gdal_array, gdalconst, osr

# Explicitly use GDAL exceptions
gdal.UseExceptions()

# Projection information
geog_crs = 'EPSG:4326'  # Geographic projection
prj_crs = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire'
datadir = os.path.join(maindir,'Aim2/data/spatial/raw/VIIRS/')
dataoutdir = os.path.join(maindir,'Aim2/data/spatial/mod/VIIRS/')

# File path information
print("Success !")

Success !


In [2]:
# Function to convert swath to grid

def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)


def utmLookup(lat, lon):
    utm = str((math.floor((lon + 180) / 6) % 60) + 1)
    if len(utm) == 1:
        utm = '0' + utm
    if lat >= 0:
        epsg_code = '326' + utm
    else:
        epsg_code = '327' + utm
    return epsg_code


def subset_array(array, areaDef, boundingCoords):
    """ Subset the array using the fire perimeter coordinates """
    min_lon, min_lat = np.min(boundingCoords[:, 0]), np.min(boundingCoords[:, 1])
    max_lon, max_lat = np.max(boundingCoords[:, 0]), np.max(boundingCoords[:, 1])
    
    # Transform the coordinates to array indices
    col_start = int((min_lon - areaDef.area_extent[0]) / areaDef.pixel_size_x)
    col_end = int((max_lon - areaDef.area_extent[0]) / areaDef.pixel_size_x)
    row_start = int((areaDef.area_extent[3] - max_lat) / areaDef.pixel_size_y)
    row_end = int((areaDef.area_extent[3] - min_lat) / areaDef.pixel_size_y)

    return array[row_start:row_end, col_start:col_end]


def create_global_grid(resolution=375):
    """ Creates a global sinusoidal grid at the given resolution """
    # Sinusoidal projection parameters
    radius = 6371007.181  # Radius of Earth in meters
    pixel_size = resolution  # Pixel size in meters

    # Calculate the number of columns and rows
    cols = int((2 * np.pi * radius) / pixel_size)
    rows = int((np.pi * radius) / pixel_size)

    # Define the area extent in meters for the sinusoidal projection
    area_extent = (-radius * np.pi, -radius * 0.5 * np.pi, radius * np.pi, radius * 0.5 * np.pi)

    # Create the sinusoidal projection string
    proj_str = (
        'PROJCS["unnamed",'
        'GEOGCS["Unknown datum based upon the custom spheroid", '
        'DATUM["Not specified (based on custom spheroid)", '
        'SPHEROID["Custom spheroid",6371007.181,0]], '
        'PRIMEM["Greenwich",0],'
        'UNIT["degree",0.0174532925199433]],'
        'PROJECTION["Sinusoidal"], '
        'PARAMETER["longitude_of_center",0], '
        'PARAMETER["false_easting",0], '
        'PARAMETER["false_northing",0], '
        'UNIT["Meter",1]]'
    )

    # Create the area definition using pyresample
    area_def = geom.AreaDefinition(
        'sinusoidal', 'Global Sinusoidal', proj_str,
        {
            'proj': 'sinu',
            'lon_0': 0,
            'x_0': 0,
            'y_0': 0,
            'a': radius,
            'b': radius,
            'units': 'm'
        },
        cols, rows, area_extent
    )

    return area_def
    

def export_geotiff(da_array, out_file_name, geotransform, epsg, na_data_val):
    """ Exports a data array as GeoTIFF with specified paramters """
    """
    Args:
        - da_array: data array to be exported as a geotiff
        - out_file_name: the file path (full) for the exported geotiff
        - geotransform: the defined geotransformation to be applied
        - epsg: output coordinate reference system
        - no_data_val: no data value to write 
    Returns:
        - Exported GeoTIFF
    """
    height, width = da_array.shape  # Define array dimensions
    # Set up the driver and output file
    driv = gdal.GetDriverByName('GTiff')
    dataType = gdal_array.NumericTypeCodeToGDALTypeCode(da_array.dtype)
    d = driv.Create(out_file_name, width, height, 1, dataType)
    d.SetGeoTransform(geotransform)
    # Define spatial reference system
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(int(epsg))
    d.SetProjection(srs.ExportToWkt())
    band = d.GetRasterBand(1)
    band.WriteArray(da_array)
    # Define fill value if it exists, if not, set to mask fill value
    if na_data_val is not None and na_data_val != 'NaN':
        band.SetNoDataValue(na_data_val)
    else:
        band.SetNoDataValue(np.nan)
    # Flush the cash
    band.FlushCache()
    d, band = None, None
    

def viirs_swath2grid(fireDA, geoDA, shortName, sdsName, boundingCoords, out_dir):
    """ Converts VIIRS AFD NetCDF SDS to grid and exports as GeoTIFF """
    
    """
    Args:
        - fireDA: The NetCDF file containing the fire information (*.nc)
        - geoDA: The corresponding geolocation file (*.h5)
        - shortName: The short name for the data product (e.g., VNP14IMG, VJ114IMG)
        - sdsName: The name for the Science Dataset (SDS) (e.g., FP_power)
        - geomBounds: the bounding geometry to create the output spatial array
        - geomCoords: list of coordinate pairs, used to filter the data array
    Returns:
        - Spatial (projected) array for the given SDS and bounding geometry
    """

    #############################################################
    # Open the geolocation data and read contents (lat/lon SDS) #

    # Get the original file name 
    geoName = os.path.basename(geoDA).split('.nc')[0]

    # Read the geolocation datasets (lat/lon)
    geo_ds = Dataset(geoDA, 'r')
    geoloc_da = geo_ds.groups['geolocation_data'] # where the geolocation data is stored
    # Retrieve the coordinate SDS
    lat = np.array(geoloc_da.variables['latitude'][:]).astype(np.float32)
    lon = np.array(geoloc_da.variables['longitude'][:]).astype(np.float32)
    print(f"latGEO shape: {lat.shape}\nlonGEO shape: {lon.shape}\nData Type: {type(lat)}")
    # Store the dimensions for later
    dims = lat.shape # shape of the swath coordinate array
    
    # Get the middle swath latlon
    midLat, midLon = np.mean(lat), np.mean(lon) 

    # Identify the UTM Zone of the middle swath (NOT IMPLEMENTED YET)
    utm_zone = utmLookup(midLat, midLon)
    print(f"UTM Zone of middle swath: {utm_zone}")
    
    #####################################################
    # Load data from NetCDF file (VNP14IMG or VJ114IMG) #
    
    afd_ds = Dataset(fireDA, 'r')

    # Check if it is day or night
    daynight = afd_ds.getncattr('DayNightFlag')

    # Grab the fire mask (full array)
    fire_mask = afd_ds.variables['fire mask'][:]
    # Grab the Fire Pixel (FP) sparse data arrays 
    FP_power = afd_ds.variables['FP_power'][:]
    FP_latitude = afd_ds.variables['FP_latitude'][:]
    FP_longitude = afd_ds.variables['FP_longitude'][:]

    del afd_ds # clean up, we have the arrays we need

    # Debugging prints
    print(f"FP_power shape: {FP_power.shape}") # see the sparse array
    print(f"FP_latitude shape: {FP_latitude.shape}")
    print(f"FP_longitude shape: {FP_longitude.shape}")
    print(f"Fire Mask shape: {fire_mask.shape}") # see the full array

    ##################################################################
    # Create the swath definition, area definition, and geotransform #
    
    # Swath Definition from latlon arrays
    swathDef = geom.SwathDefinition(lons=lon, lats=lat) # from 'pyresample' geom

    # Create area definition using coordinate arrays and projection information
    # Use info from aeqd bbox to calculate output cols/rows/pixel size
    epsgConvert = pyproj.Proj("+proj=aeqd +lat_0={} +lon_0={}".format(midLat, midLon))
    llLon, llLat = epsgConvert(np.min(lon), np.min(lat), inverse=False)
    urLon, urLat = epsgConvert(np.max(lon), np.max(lat), inverse=False)
    areaExtent = (llLon, llLat, urLon, urLat)
    cols = int(round((areaExtent[2] - areaExtent[0])/375))  # 375 m pixel size
    rows = int(round((areaExtent[3] - areaExtent[1])/375))
    '''Use no. rows and columns generated above from the aeqd projection
                to set a representative number of rows and columns, which will then be translated
                to degrees below, then take the smaller of the two pixel dims to determine output size'''
    epsg, proj, pName = '4326', 'longlat', 'Geographic'
    llLon, llLat, urLon, urLat = np.min(lon), np.min(lat), np.max(lon), np.max(lat)
    areaExtent = (llLon, llLat, urLon, urLat)
    projDict = pyproj.CRS("epsg:4326") # geographic coordinate projection
    areaDef = geom.AreaDefinition(epsg, pName, proj, projDict, cols, rows, areaExtent)
    ps = np.min([areaDef.pixel_size_x, areaDef.pixel_size_y])  # Square pixels

    # Now, recalculate the cols, rows, and area definition based on the pixel dimensions in degrees
    cols = int(round((areaExtent[2] - areaExtent[0])/ps))  # Calculate the output cols
    rows = int(round((areaExtent[3] - areaExtent[1])/ps))  # Calculate the output rows
    areaDef = geom.AreaDefinition(epsg, pName, proj, projDict, cols, rows, areaExtent)
    print(f"Pixel Dims: {ps};\nNumber of columns: {cols};\nNumber of rows: {rows}\nArea definition shape: {areaDef.shape}")

    # Gather the geotransform definition
    gt = [areaDef.area_extent[0], ps, 0, areaDef.area_extent[3], 0, -ps]
    
    #####################################################
    # Perform nearest neighbor sampling (SWATH to GRID) #
    
    # Get the neighbor info using radius of influence 3x pixel dims
    index, outdex, indexArr, distArr = kdt.get_neighbour_info(swathDef, areaDef, 375, neighbours=1)
    # Perform kdtree resampling (swath 2 grid conversion) --- for the fire mask
    fv = -9999 # for the uint8 data type of the fire mask
    sdGEO = kdt.get_sample_from_neighbour_info('nn', areaDef.shape, fire_mask, index, outdex, indexArr, fill_value=fv)

    # Create a full grid for FP_power based on the fire mask grid using pyresample's kd_tree.resample_nearest
    fv = np.nan
    # Create a new swath definition
    swathDef_fire = geom.SwathDefinition(lons=FP_longitude, lats=FP_latitude) # fll is within the geometry bounds
    index, outdex, indexArr, distArr = kdt.get_neighbour_info(swathDef_fire, areaDef, 375, neighbours=1)
    FP_power_grid = kdt.get_sample_from_neighbour_info('nn', areaDef.shape, FP_power, index, outdex, indexArr, fill_value=fv)

    ###################################################################################
    # Mask the FRP grid to include only active fire pixels (nominal or high confidence)
    fire_pixels = (sdGEO == 8) | (sdGEO == 9)
    FP_power_grid[~fire_pixels] = np.nan # Set areas outside the active fire detections as NaN
    # Further mask the array by a bounding geometry

    ########################################################################
    # Define a global sinusoidal grid and resample the swath grid to it    #

    index, outdex, indexArr, distArr = kdt.get_neighbour_info(areaDef, global_area_def, 375, neighbours=1)
    sdGEO_res = kdt.get_sample_from_neighbour_info('nn', global_area_def.shape, sdGEO, index, outdex, indexArr, fill_value=fv)
    # FP_power_grid_res = kdt.get_sample_from_neighbour_info('nn', global_area_def.shape, FP_power_grid, index, outdex, indexArr, fill_value=np.nan)


    ############################################
    # Set up the GeoTIFF export for day or night
    outDir = os.path.join(out_dir, f'georeferenced/{shortName}/{daynight}')
    # Check the directory exists, make it if not
    if not os.path.exists(outDir):
        os.makedirs(outDir)

    # Set up output name
    identifier_ = identifier.replace(".", "_")
    sdsName = sdsName.replace(" ", "_")
    platform_datetime = identifier_.split('_')[0] + "_" + identifier_.split('_')[1] + "_" + identifier_.split('_')[2]
    outName = os.path.join(outDir, sdsName + '_' + platform_datetime + '.tif')
    print(f"\noutput file:\n{outName}\n")

    # Export the GeoTIFF !!!
    export_geotiff(FP_power_grid, outName, gt, epsg, fv)

    outName = outName.replace("FP_power", "fire_mask")
    export_geotiff(sdGEO_res, outName, gt, epsg, fv)


def get_coords(geom, buffer):
    """ Returns the bounding box coordinates for a given geometry(ies) and buffer """
    _geom = geom.copy()
    _geom['geometry'] = _geom.geometry.buffer(buffer)
    bounds = _geom.to_crs(geog_crs).unary_union.envelope # make sure it is in geographic coordinates
    coords = list(bounds.exterior.coords)

    del _geom, bounds
    return coords


def calculate_max_frp(frp_stack, common_area_def):
    dates = sorted(frp_stack.keys())
    frp_data_arrays = [frp_stack[date] for date in dates]
    # Convert the list of arrays to an xarray DataArray
    frp_stack_da = xr.DataArray(frp_data_arrays, dims=['time', 'y', 'x'])
    # Resample to a common pixel alignment
    common_frp_stack = frp_stack_da.resample({'time': '1D'}).max(dim='time', skipna=True)
    # Calculate the maximum FRP across the time series
    max_frp = common_frp_stack.max(dim='time', skipna=True)
    return max_frp

print("Function to process VIIRS NetCDF files is ready to use!")


Function to process VIIRS NetCDF files is ready to use!


In [3]:
# Testing for one fire
testDir = os.path.join(datadir,'FIRED_3518')

# Get list of fire data files
vnp_files = list_files(testDir,"VNP14*.nc",recursive=True)
vj1_files = list_files(testDir,"VJ114*.nc",recursive=True)

# Grab the geolocation data
geo_files = list_files(testDir,"*03IMG*.nc",recursive=True)

print(f"Length of VNP14IMG: {len(vnp_files)}\nLength of VJ114IMG: {len(vj1_files)}\nLength of Geolocation files: {len(geo_files)}")

print(vnp_files[0])
print(vj1_files[0])
print(geo_files[0])

# Create a dictionary to store the file paths
datadict = {
    'VNP14IMG': vnp_files,
    'VJ114IMG': vj1_files
}

Length of VNP14IMG: 17
Length of VJ114IMG: 16
Length of Geolocation files: 33
/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_3518/VNP14IMG/VNP14IMG.A2021242.2048.002.2024074102039.nc
/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_3518/VJ114IMG/VJ114IMG.A2021239.0936.002.2024081210828.nc
/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_3518/VNP03IMG/VNP03IMG.A2021240.2130.002.2021264132733.nc


In [4]:
# Load fire data and create a dictionary with bounding coordinates
fires_path = os.path.join(maindir,'Aim2/data/spatial/mod/FIRED/fired_events_west_aspen.gpkg')
fires = gpd.read_file(fires_path)
print(fires.columns)
print(len(fires))

Index(['fired_id', 'ig_date', 'ig_day', 'ig_month', 'ig_year', 'last_date',
       'event_dur', 'tot_pix', 'tot_ar_km2', 'fsr_px_dy', 'fsr_km2_dy',
       'mx_grw_px', 'mn_grw_px', 'mu_grw_px', 'mx_grw_km2', 'mn_grw_km2',
       'mu_grw_km2', 'mx_grw_dte', 'x', 'y', 'ig_utm_x', 'ig_utm_y', 'lc_code',
       'lc_mode', 'lc_name', 'lc_desc', 'lc_type', 'eco_mode', 'eco_name',
       'eco_type', 'tot_perim', 'pct_aspen', 'geometry'],
      dtype='object')
102


In [5]:
# Create a dictionary to store fire bounding coordinates
# These can be used to subset the arrays before exporting the data
coords_dict = {}
buffer = 375 

for index, row in fires.iterrows():
    fire_id = row['fired_id']
    perim = fires.loc[fires['fired_id'] == fire_id]
    coords = get_coords(perim, buffer)
    coords_dict[fire_id] = coords

# Print the dictionary to verify
first = next(iter(coords_dict.items()))
print(f"FIRED_ID: {first[0]}, \nBounding Coordinates: \n{first[1]}")

FIRED_ID: 3518, 
Bounding Coordinates: 
[(-113.4870457742665, 37.31747928447832), (-113.45119936806327, 37.31747928447832), (-113.45119936806327, 37.336690972515356), (-113.4870457742665, 37.336690972515356), (-113.4870457742665, 37.31747928447832)]


In [6]:
# Create a the global sinusoidal grid
global_area_def = create_global_grid()
print(global_area_def.shape)

(53373, 106747)


In [ ]:
t0 = time.time()

dat = 'FP_power' # the SDS we are extracting ...

fired_id = '3518'
coords_ = coords_dict[fired_id]

out_dir = testDir

for short_name, fpaths in datadict.items():
    print(f"Processing NetCDF files for {short_name}")
    # Retrieve the geolocations files corresponding to the short name
    sh_code = short_name[:3] # the platform code (e.g., 'VNP')
    _geo_files = [gf for gf in geo_files if sh_code in os.path.basename(gf)]
    print(f"There are {len(_geo_files)} associated geolocation files ...")
    for fp in fpaths:
        identifier = os.path.basename(fp)[:-3]
        print(identifier)

        # Open the NetCDF file
        ds = Dataset(fp, 'r', format='NETCDF4')  # Read in VIIRS AFD file

        # Create a list of all SDS inside of the .nc file
        ecoSDS = list(ds.variables.keys())

        del ds # clean up !

        # Find the matching geolocation file from the file list
        parts = identifier.split('.')
        date_time_part = '.'.join(parts[1:3])  # Extract date-time parts for the VNP Version 002
        geo_identifier = sh_code + '03IMG' + '.' + date_time_part
        geo = [geo_link for geo_link in _geo_files if geo_identifier in os.path.basename(geo_link)][0]        
        print(geo)

        ###################################
        # Now apply our processing function

        try:
            viirs_swath2grid(
                fireDA=fp, 
                geoDA=geo, 
                shortName=short_name, 
                sdsName=dat,
                boundingCoords=coords_, 
                out_dir=out_dir
            )
        except Exception as e:
            print(f"Skipping granule {identifier}\n{e}")
            # traceback.print_exc()  # This will print the full traceback
            continue  # continue to the next granule
        
        print('Time to complete granule:', time.time() - t0)
        print("\n")
        print("---------------------------------------------")


Processing NetCDF files for VNP14IMG
There are 17 associated geolocation files ...
VNP14IMG.A2021242.2048.002.2024074102039
/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_3518/VNP03IMG/VNP03IMG.A2021242.2048.002.2021264144156.nc
latGEO shape: (6464, 6400)
lonGEO shape: (6464, 6400)
Data Type: <class 'numpy.ndarray'>
UTM Zone of middle swath: 32611
FP_power shape: (1659,)
FP_latitude shape: (1659,)
FP_longitude shape: (1659,)
Fire Mask shape: (6464, 6400)
Pixel Dims: 0.0034526084770449017;
Number of columns: 11060;
Number of rows: 7308
Area definition shape: (7308, 11060)
